### Import das dependências

In [21]:
import os

import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme()

### 1. Importar o dataset - "Número de ordens de serviço remotas realizadas"

In [22]:
data_path = '../data/' if os.path.exists('../data/') else 'https://raw.githubusercontent.com/kreativermario/Projeto-DECD/master/data/'

ordens_servico_path = data_path + '15-ordens-de-servico.csv'

df_ordens_servico = pd.read_csv(ordens_servico_path, delimiter=';')

### 2. Ver o dataset

In [23]:
df_ordens_servico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194181 entries, 0 to 194180
Data columns (total 9 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Ano                            194181 non-null  int64 
 1   Mes                            194181 non-null  int64 
 2   data                           194181 non-null  object
 3   Distrito                       194181 non-null  object
 4   Concelho                       194181 non-null  object
 5   Freguesia                      194181 non-null  object
 6   Tipo de Serviço                194181 non-null  object
 7   Ordens Serviço Realizadas (#)  194181 non-null  int64 
 8   CodDistritoConcelhoFreguesia   194181 non-null  object
dtypes: int64(3), object(6)
memory usage: 13.3+ MB


1. Verificar se há elementos null

In [24]:
# Verificar nulls
null_counts = df_ordens_servico.isnull().sum()

# Ver nulls por coluna
print("Número de nulls por coluna:")
print(null_counts)

df_ordens_servico.head(15)

Número de nulls por coluna:
Ano                              0
Mes                              0
data                             0
Distrito                         0
Concelho                         0
Freguesia                        0
Tipo de Serviço                  0
Ordens Serviço Realizadas (#)    0
CodDistritoConcelhoFreguesia     0
dtype: int64


,Ano,Mes,data,Distrito,Concelho,Freguesia,Tipo de Serviço,Ordens Serviço Realizadas (#),CodDistritoConcelhoFreguesia
0,2022,8,2022-08,Coimbra,Soure,Soure,Redução de Potência Contratada,51,061509
1,2022,8,2022-08,Coimbra,Tábua,União das freguesias de Covas e Vila Nova de O...,Configuração de EMI,1,061617
2,2022,8,2022-08,Coimbra,Vila Nova de Poiares,Arrifana,Reposição de Potência Contratada,1,061701
3,2022,8,2022-08,Coimbra,Vila Nova de Poiares,Poiares (Santo André),Serviço de Corte e Re-Ligação,6,061703
4,2022,8,2022-08,Coimbra,Vila Nova de Poiares,São Miguel de Poiares,Reposição de Potência Contratada,1,061704
5,2022,8,2022-08,Faro,Albufeira,Guia,Redução de Potência Contratada,4,080102
6,2022,8,2022-08,Faro,Albufeira,Guia,Reposição de Potência Contratada,6,080102
7,2022,8,2022-08,Faro,Alcoutim,União das freguesias de Alcoutim e Pereiro,Serviço de Corte e Re-Ligação,4,080206
8,2022,8,2022-08,Faro,Aljezur,Aljezur,Serviço de Corte e Re-Ligação,7,080301
9,2022,8,2022-08,Faro,Aljezur,Rogil,Redução de Potência Contratada,2,080304


### 3. Tratamento e agregação dos dados

1. Remover atributos como a data, CodDistritoConcelhoFreguesia
2. Agrupar por concelhos e remover a freguesia
3. Agrupar por semestres e anos

1. Remover atributos como a data, CodDistritoConcelhoFreguesia

In [25]:
# Remover atributos como a data e código, distrito
df_ordens_servico.drop(columns=['data', 'CodDistritoConcelhoFreguesia', 'Distrito'], inplace=True)
df_ordens_servico.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194181 entries, 0 to 194180
Data columns (total 6 columns):
 #   Column                         Non-Null Count   Dtype 
---  ------                         --------------   ----- 
 0   Ano                            194181 non-null  int64 
 1   Mes                            194181 non-null  int64 
 2   Concelho                       194181 non-null  object
 3   Freguesia                      194181 non-null  object
 4   Tipo de Serviço                194181 non-null  object
 5   Ordens Serviço Realizadas (#)  194181 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 8.9+ MB


2. Agrupar por concelhos e remover a freguesia

In [35]:
df_agrupado = df_ordens_servico.groupby(['Concelho', 'Tipo de Serviço', 'Ano', 'Mes']).sum().reset_index()
df_agrupado.drop(columns=['Freguesia'], inplace=True)
df_agrupado.head()

,Concelho,Tipo de Serviço,Ano,Mes,Ordens Serviço Realizadas (#)
0,Abrantes,Configuração de EMI,2022,3,4
1,Abrantes,Configuração de EMI,2022,4,15
2,Abrantes,Configuração de EMI,2022,5,6
3,Abrantes,Configuração de EMI,2022,6,10
4,Abrantes,Configuração de EMI,2022,7,8


3. Verificar o número de linhas

In [36]:
# Contagem de linhas antes do agrupamento
num_linhas_antes = len(df_ordens_servico)

# Contagem de linhas depois do agrupamento
num_linhas_depois = len(df_agrupado)

# Exibição das contagens
print("Número de linhas antes do agrupamento:", num_linhas_antes)
print("Número de linhas depois do agrupamento:", num_linhas_depois)

df_agrupado.info()

df_agrupado.head(15)


Número de linhas antes do agrupamento: 194181
Número de linhas depois do agrupamento: 36598
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36598 entries, 0 to 36597
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Concelho                       36598 non-null  object
 1   Tipo de Serviço                36598 non-null  object
 2   Ano                            36598 non-null  int64 
 3   Mes                            36598 non-null  int64 
 4   Ordens Serviço Realizadas (#)  36598 non-null  int64 
dtypes: int64(3), object(2)
memory usage: 1.4+ MB


,Concelho,Tipo de Serviço,Ano,Mes,Ordens Serviço Realizadas (#)
0,Abrantes,Configuração de EMI,2022,3,4
1,Abrantes,Configuração de EMI,2022,4,15
2,Abrantes,Configuração de EMI,2022,5,6
3,Abrantes,Configuração de EMI,2022,6,10
4,Abrantes,Configuração de EMI,2022,7,8
5,Abrantes,Configuração de EMI,2022,8,6
6,Abrantes,Configuração de EMI,2022,9,4
7,Abrantes,Configuração de EMI,2022,10,32
8,Abrantes,Configuração de EMI,2022,11,268
9,Abrantes,Configuração de EMI,2022,12,382


4. Verificar se está tudo bem

In [37]:
# Para a coleção de Consumos Mensais por Concelho
df_consumos_lisboa = df_agrupado[df_agrupado['Concelho'] == 'Lisboa']
df_consumos_lisboa.info()

<class 'pandas.core.frame.DataFrame'>
Index: 159 entries, 14579 to 14737
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Concelho                       159 non-null    object
 1   Tipo de Serviço                159 non-null    object
 2   Ano                            159 non-null    int64 
 3   Mes                            159 non-null    int64 
 4   Ordens Serviço Realizadas (#)  159 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 7.5+ KB


In [29]:
df_consumos_lisboa

,Concelho,Tipo de Serviço,Ano,Mes,Ordens Serviço Realizadas (#)
14579,Lisboa,Configuração de EMI,2022,3,12
14580,Lisboa,Configuração de EMI,2022,4,23
14581,Lisboa,Configuração de EMI,2022,5,17
14582,Lisboa,Configuração de EMI,2022,6,12
14583,Lisboa,Configuração de EMI,2022,7,20
...,...,...,...,...,...
14733,Lisboa,Serviço de Corte e Re-Ligação,2023,11,734
14734,Lisboa,Serviço de Corte e Re-Ligação,2023,12,544
14735,Lisboa,Serviço de Corte e Re-Ligação,2024,1,660
14736,Lisboa,Serviço de Corte e Re-Ligação,2024,2,395


5. Guardar num ficheiro

In [41]:
file_path = data_path + 'treated/ordens_servico_agrupado_por_mes.csv'

df_agrupado.to_csv(file_path, index=False)

print("DataFrame guardado em ", file_path)


DataFrame guardado em  ../data/treated/ordens_servico_agrupado_por_mes.csv
